# Retry Methods that Fail
Unluckily `tenacity` is not compatible with `pandarallel`, thus a manual implementation of retrying is needed.  
However, the following code trying to integrate `logging` into `tenacity` might be useful later. 

In [ ]:
import os
import pathlib
import random
import logging

from tenacity import retry, stop_after_attempt, before_sleep_log 
import pandas as pd
import numpy as np

file_path = os.path.abspath('')
PROJECT_ROOT = pathlib.Path(file_path).parent.parent.resolve()
DATA_DIR = PROJECT_ROOT / "data" / "fmow_landsat"
LOG_FILE = 'retry_log.log'

logger = logging.getLogger('retry_log')
logger.setLevel(logging.INFO)

file_handler = logging.FileHandler(LOG_FILE, mode='w')
file_handler.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)

In [2]:
@retry(stop=stop_after_attempt(10), 
       before_sleep=before_sleep_log(logger, logging.INFO))
def do_something_unreliable(row):
    if random.randint(0, 10) > 9:
        raise IOError("Broken sauce, everything is hosed!!!111one")
    else:
        return "Awesome sauce!"

In [3]:
big_df = pd.DataFrame(0, index=np.arange(100), columns=['A'])
big_df.apply(do_something_unreliable, axis=1)

0     Awesome sauce!
1     Awesome sauce!
2     Awesome sauce!
3     Awesome sauce!
4     Awesome sauce!
           ...      
95    Awesome sauce!
96    Awesome sauce!
97    Awesome sauce!
98    Awesome sauce!
99    Awesome sauce!
Length: 100, dtype: object

In [4]:
if os.path.exists(LOG_FILE):
    os.remove(LOG_FILE)